In [1]:
import sqlite3
import os
from random import choices
import pandas as pd
from datetime import date

In [2]:
cartoon_dir = r'E:\TV Shows'
commercial_dir = r'E:\Bryce'
os.setcwd = r'C:\Users\Erik\Documents\GitHub\saturday_morning'
file_extensions = ['avi', 'mpg', 'mp4']
playlist_length = 6

In [3]:
conn = sqlite3.connect('cartoons.db')
c = conn.cursor()
c.execute('''SELECT name FROM sqlite_master WHERE TYPE = 'table';''')

tables = c.fetchall()
tables = [i[0] for i in tables] 

if tables is None:
    tables = []

def crawl_dirs(directory, table, c, conn):
    existing_qry = c.execute(f'''SELECT filepath FROM {table}''')
    existing_files = existing_qry.fetchall()
    existing_files = [i[0] for i in existing_files]
    for root, dirs, files in os.walk(directory, topdown=False):
        for name in files:
            if name.rsplit('.', maxsplit=1)[1] in file_extensions:
                filepath = os.path.join(root, name)
                if filepath not in existing_files:
                    last_played = 'NULL'
                    play_count = 0
                    c.execute(f'''INSERT INTO {table} VALUES (:last_played, :filepath, :play_count)''', 
                              {'last_played': last_played,
                               'filepath': filepath,
                               'play_count': play_count})
    conn.commit()

if 'cartoons' not in tables:
    c.execute('''CREATE TABLE cartoons
                 (last_accessed date, filepath text, play_count real)''')
    conn.commit()
crawl_dirs(cartoon_dir, 'cartoons', c, conn)

if 'commercials' not in tables:
    c.execute('''CREATE TABLE commercials
                 (last_accessed date, filepath text, play_count real)''')
    conn.commit()
crawl_dirs(commercial_dir, 'commercials', c, conn)

with open('playlist.txt', 'w') as playlist:
    def construct_playlist(playlist):
        playlist.write('[playlist]\n')
        playlist.write(f'NumberOfEntries={playlist_length * 3}\n')
        playlist_lines = []

        def add_to_playlist(table, playlist_lines, i):
            i += 1
            min_play_qry = c.execute(f"SELECT MIN(play_count) FROM {table}")
            min_play_count = min_play_qry.fetchone()[0]

            least_played = c.execute(f'''SELECT * FROM {table} WHERE play_count = :min_play''', 
                                     {'min_play': min_play_count})
            least_df = pd.DataFrame(least_played.fetchall(), 
                                columns=['last_played', 'filepath', 'play_count'])

            chosen_index = choices(least_df.index)
            chosen_row = pd.Series(least_df.iloc[chosen_index].to_dict('records')[0])
            filepath = chosen_row.loc['filepath']
            play_count = int(chosen_row.loc['play_count']) + 1

            sql = f'''UPDATE {table}
                      SET last_accessed = :today ,
                          play_count = :play_count
                      WHERE filepath = :filepath;'''
            c.execute(sql, {'today': date.today(), 'play_count': play_count, 'filepath': filepath})
            conn.commit()
            playlist_lines += [f'''File{i}={filepath}\n''']
            return playlist_lines, i

        for i in [i*3 for i in range(playlist_length)]:
            playlist_lines, i = add_to_playlist('cartoons', playlist_lines, i)
            playlist_lines, i = add_to_playlist('commercials', playlist_lines, i)
            playlist_lines, i = add_to_playlist('commercials', playlist_lines, i)

        playlist.writelines(playlist_lines)
        playlist.close()
        
    construct_playlist(playlist)

conn.close()